In [1]:
import re
import math

In [12]:
def getwords(doc):
    '''
        Returns words in form of dictionary
    '''
    splitter = re.compile('\\W*')
    #split words by non alpha characters
    words = [s.lower() for s in splitter.split(doc) if len(s)>2 and len(s)<20 ]
    
    # return unique set of words only
    return dict([(w,1) for w in words])

In [14]:
class classifier:
    def __init__(self,getfeatures,filename=None):
        #counts of feature/category combinatios
        self.fc={}
        #counts of documents in each category
        self.cc={}
        self.getfeatures = getfeatures
    
    # Increase the count of a feature/category pair
    def incf(self,f,cat):
        self.fc.setdefault(f,{})
        self.fc[f].setdefault(cat,0)
        self.fc[f][cat]+=1
        
    # Increase the count of a category
    def incc(self,cat):
        self.cc.setdefault(cat,0)
        self.cc[cat]+=1
    
    # The number of times a feature has appeared in category
    def fcount(self,f,cat):
        if f in self.f and cat in self.cat:
            return float(self.fc[f][cat])
        return 0.0
    
    # The number of items in a category
    def catcount(self,cat):
        if cat in self.cc:
            return float(self.cc[cat])
        return 0
    
    # The total number of items
    def totalcount(self):
        return sum(self.cc.values())
    
    # The list of all categories
    def categories(self):
        return self.cc.keys()
    
    def train(self,item,cat):
        features = self.getfeatures(item)
        # increment count of every feature in this category
        for f in features:
            self.incf(f,cat)
        
        # increment count of this category
        self.incc(cat)

In [15]:
# using getwords as the feature extractor.
c1 = classifier(getwords)